In [ ]:
import os
import shutil

if os.path.exists('datasets'):
    shutil.rmtree('datasets')
    
if os.path.isfile('masked-face-datasets.zip'):
    os.remove('masked-face-datasets.zip')
    
!wget https://jarkom-ibm-cloud-2022.s3.jp-tok.cloud-object-storage.appdomain.cloud/masked-face-datasets.zip
!unzip masked-face-datasets.zip

os.rename('masked-face-datasets', 'datasets')

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Nadam
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from load_image import *
from base_model import *
from plot import *

model = model_VGG16()
model.compile(loss="categorical_crossentropy",
              optimizer=Nadam(
                learning_rate=0.001,
              ),
              metrics=["accuracy"])
model_summary = model.summary()

In [ ]:
import time
from keras.callbacks import EarlyStopping, ModelCheckpoint

earlystopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('best_model.h5',  save_best_only=True)

test_time = time.strftime('%Y%m%d_%H%M')
start_history = time.perf_counter()

epochs = 30
steps_size_train = train_data_augmented.n // train_data_augmented.batch_size
steps_size_validation = validation_data.n // validation_data.batch_size

history = model.fit(train_data_augmented,
                    steps_per_epoch=steps_size_train,
                    validation_data=validation_data,
                    validation_steps=steps_size_validation,
                    epochs=epochs,
                    callbacks=[
                        checkpoint,
                        # earlystopping
                    ]
                    )

elapsed_history = time.perf_counter() - start_history

In [ ]:
train_eval_scores = model.evaluate(
    train_data_augmented,
    batch_size=validation_data.batch_size
    )

train_loss = train_eval_scores[0]
train_acc = round(train_eval_scores[1] * 100, 2)
print(f'Training Loss : {train_loss}')
print(f'Training Accuracy : {train_acc}%\n')

val_eval_scores = model.evaluate(
    validation_data,
    batch_size=validation_data.batch_size
    )

val_loss = val_eval_scores[0]
val_acc = round(val_eval_scores[1] * 100, 2)
print(f'Validation Loss : {val_loss}')
print(f'Validation Accuracy : {val_acc}%')

In [ ]:
import os

## Create directory to save the result
result_dir = f'results/{test_case}/{test_time}'
data_test_dir = f'{result_dir}/data_test'

if (os.path.exists(result_dir) == False):
    os.makedirs(result_dir)
        
if not os.path.exists(data_test_dir):
    os.makedirs(data_test_dir)

In [ ]:
import os
import shutil

## move best_model file
if os.path.isfile('best_model.h5'):
    shutil.move('best_model.h5', f'{result_dir}')

## Write History Time and Evaluate into a txt file
f = open(f'{result_dir}/result_data.txt', 'a')

f.write('- fit time \t: %.3f seconds' % elapsed_history)
f.write('\n\n')

f.write(f'- image_width \t: {img_width}\n')
f.write(f'- image_height \t: {img_height}\n')
f.write('\n\n')

f.write(f'- batch size dataset train: \t{train_data_augmented.batch_size}\n')
f.write(f'- batch size dataset val: \t{validation_data.batch_size}\n')
f.write('\n\n')

f.write(f'Fit model\n')
f.write(f'- epoch \t: {epochs}\n')
f.write('\n\n')

f.write(f'- train_loss \t: {train_loss}\n')
f.write(f'- train_acc \t: {train_acc}%\n')
f.write('\n\n')

f.write(f'- val_loss \t: {val_loss}\n')
f.write(f'- val_acc \t: {val_acc}%\n')
f.close()

# Save the model
model_filename = "model"
model_json = model.to_json()
with open(f'{result_dir}/{model_filename}.json', "w") as json_file:
    json_file.write(f'{result_dir}/{model_json}')

model.save(f'{result_dir}/{model_filename}.h5', save_format='h5')

In [ ]:
loss_plot = plot_loss_curves(history)
loss_plot.savefig(f'{result_dir}/loss_graph.jpg', bbox_inches='tight')

In [ ]:
acc_plot = plot_accuracy_curves(history)
acc_plot.savefig(f'{result_dir}/acc_graph.jpg', bbox_inches='tight')

In [ ]:
from keras.preprocessing import image as krimg
import matplotlib.image as mpimpg
import glob
import os
import ntpath
    
faceimagedir = 'datasets/'+ test_case +'/test'
best_model = load_model(f'{result_dir}/best_model.h5')

images_paths = []
for img_path in glob.glob(faceimagedir + '/*.jpg'):
    images_paths.append(img_path)
    
for i, image in enumerate(images_paths):
    img = krimg.load_img(image, target_size=(img_width, img_height))
    image_filename = ntpath.basename(image)

    image_to_predict = np.expand_dims(img, axis=0)
    predictions = best_model.predict(image_to_predict)
    classes = np.argmax(predictions, axis = 1)
    prediction_name = class_names[classes][0]
    
    if prediction_name:
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(prediction_name)
        plt.imshow(img)
        plt.savefig(f'{result_dir}/data_test/{image_filename}', bbox_inches='tight')

In [ ]:
img, label = train_data_augmented.next()

plt.figure(figsize=(15, 8))

# generate batch of images
for i in range(0, 5):
    plt.subplot(1, 5, 1 + i)
    plt.imshow(img[i])

plt.savefig(f'{result_dir}/augmentasi_data.jpg', bbox_inches='tight')